In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import pickle
import mlflow
import xgboost as xgb

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/Users/abhinav/Documents/workbench/personal/mlops/02-Experiment Tracking and '
 'Model Management/mlflow-scenario-examples/mlruns/1'), creation_time=1685004904968, experiment_id='1', last_update_time=1685004904968, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(file_name):
    df = pd.read_parquet(file_name)
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >=1) & (df.duration <=60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype('str')
    return df 


In [5]:
df_train = read_dataframe("../../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../../data/green_tripdata_2021-02.parquet")

In [6]:
df_train['PU_DO'] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val['PU_DO'] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [7]:
categorical = ["PU_DO"] #['PULocationID', 'DOLocationID']
numerical = ['trip_distance']


dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dict = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)


In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [10]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

9.233436225720547

In [11]:
ridge = Ridge(alpha=0.001)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.511008883160363

In [12]:
with open("../../models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [13]:
with open("../../models/lasso_reg.bin", "wb") as f_out:
    pickle.dump((dv, lasso), f_out)

# Using MLFlow - Plain Vanilla

In [13]:
with mlflow.start_run(run_name="dummy_run_name_1"):
    mlflow.set_tag("developer", "abhinav")

    mlflow.log_param("train_data_path", "../../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid_data_path", "../../data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("../../models/lasso_reg.bin", "wb") as f_out:
        pickle.dump((dv, lasso), f_out)

    mlflow.log_artifact(local_path="../../models/lasso_reg.bin", artifact_path="models_pickle")
    

# Using MLFlow - Using Hyperparameter Tunning

In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [24]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}


In [25]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[15:43:22] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.27586                          
[1]	validation-rmse:17.58724                          
[2]	validation-rmse:16.10338                          
[3]	validation-rmse:14.80131                          
[4]	validation-rmse:13.66323                          
[5]	validation-rmse:12.67052                          
[6]	validation-rmse:11.80792                          
[7]	validation-rmse:11.06133                          
[8]	validation-rmse:10.41608                          
[9]	validation-rmse:9.86095                           
[10]	validation-rmse:9.38606                          
[11]	validation-rmse:8.97882                          
[12]	validation-rmse:8.63205                          
[13]	validation-rmse:8.33706                          
[14]	validation-

KeyboardInterrupt: 

## Training on best Hyperparameter

In [26]:
params = {
    'learning_rate': 0.15502423465877088,
    "max_depth":17,
    'min_child_weight': 0.5663660157121866,
    "objective":"reg:linear",
    "reg_alpha":0.021197961069748555,
    "reg_lambda":0.0031740619485586376,
    "seed":42,
}

mlflow.xgboost.autolog()
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )


2023/05/25 16:02:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3fe5f972ec014a97b2dd9933fc08a9bd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[16:02:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.45055
[1]	validation-rmse:16.17746
[2]	validation-rmse:14.30174
[3]	validation-rmse:12.76977
[4]	validation-rmse:11.52182
[5]	validation-rmse:10.51390
[6]	validation-rmse:9.70482
[7]	validation-rmse:9.06063
[8]	validation-rmse:8.54726
[9]	validation-rmse:8.14683
[10]	validation-rmse:7.82907
[11]	validation-rmse:7.58170
[12]	validation-rmse:7.38580
[13]	validation-rmse:7.23209
[14]	validation-rmse:7.11227
[15]	validation-rmse:7.01496
[16]	validation-rmse:6.94113
[17]	validation-rmse:6.88032
[18]	validation-rmse:6.83001
[19]	validation-rmse:6.79124
[20]	validation-rmse:6.75970
[21]	validation-rmse:6.73439
[22]	validation-rmse:6.71320
[23]	validation-rmse:6.69534
[24]	validation-rmse:6.67985
[25]	validation-rmse:6.66756
[26]	validation-rmse:6.65668
[27]	va

2023/05/25 16:02:44 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/05/25 16:02:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/abhinav/miniforge3/envs/ml/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


# Logging Models

In [18]:

with mlflow.start_run():
    mlflow.xgboost.autolog(disable=True)

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    params = {
        'learning_rate': 0.15502423465877088,
        "max_depth":17,
        'min_child_weight': 0.5663660157121866,
        "objective":"reg:linear",
        "reg_alpha":0.021197961069748555,
        "reg_lambda":0.0031740619485586376,
        "seed":42,
    }

    mlflow.log_params(params)

    booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # Save the preprocessor
    with open("../../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("../../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[16:53:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.45055
[1]	validation-rmse:16.17746
[2]	validation-rmse:14.30174
[3]	validation-rmse:12.76977
[4]	validation-rmse:11.52182
[5]	validation-rmse:10.51390
[6]	validation-rmse:9.70482
[7]	validation-rmse:9.06063
[8]	validation-rmse:8.54726
[9]	validation-rmse:8.14683
[10]	validation-rmse:7.82907
[11]	validation-rmse:7.58170
[12]	validation-rmse:7.38580
[13]	validation-rmse:7.23209
[14]	validation-rmse:7.11227
[15]	validation-rmse:7.01496
[16]	validation-rmse:6.94113
[17]	validation-rmse:6.88032
[18]	validation-rmse:6.83001
[19]	validation-rmse:6.79124
[20]	validation-rmse:6.75970
[21]	validation-rmse:6.73439
[22]	validation-rmse:6.71320
[23]	validation-rmse:6.69534
[24]	validation-rmse:6.67985
[25]	validation-rmse:6.66756
[26]	validation-rmse:6.65668
[27]	va

# Loading a Logged Model

## As python function

In [19]:

logged_model = 'runs:/c9f18d06bead4f3b8a66837015e5081b/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2023/05/25 16:56:14 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[16:56:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: c9f18d06bead4f3b8a66837015e5081b

## As XGBOOST Flavour

In [21]:
xgboost_model = mlflow.xgboost.load_model(model_uri=logged_model)
xgboost_model

[16:57:44] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


# Make Predictions

In [22]:
y_pred = xgboost_model.predict(valid)
y_pred

array([16.442123,  7.159368, 14.043639, ..., 13.531493,  7.159368,
        8.343674], dtype=float32)